In [ ]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

# Title
[]()
Copied code from https://github.com/mukulpatnaik/researchgpt/blob/main/main.py

In [139]:
import pickle
from datetime import datetime
def save_csv(df,filename,path=None,append_version=False):
    """
    Export dataframe to CSV.
    Parameters:
    - df: Dataframe variable name.
    - filename: Root of the filename.
    - filepath (raw string): Use the format r'<path>'. If None, file is saved in same director.
    - append_version (bool): If true, append date and time to end of filename.
    """
    if path:
        path = f'{path}/'.replace('\\','/')
    if append_version == True:
        filename+=datetime.now().strftime('%Y-%m-%d_%H%M')
    df.to_csv(path+filename+'.csv')
    print('File saved: ',path+filename+'.csv')
    print('Time completed:', datetime.now())


def savepickle(model,filename, ext='sav', path=None,append_version=False):
    """
    Export object as a pickle.
    Parameters:
    - model: Model variable name.
    - filename: Root of the filename.
    - extension: Extension to append (do not include dot as it will be added)
    - filepath (raw string): Use the format r'<path>'. If None, file is saved in same director.
    - append_version (bool): If true, append date and time to end of filename.
    """
    if path:
        path = f'{path}/'.replace('\\','/')
    if append_version == True:
        filename+=datetime.now().strftime('%Y-%m-%d_%H%M')
    with open (path+filename+'.'+ext, 'wb') as fh:
        pickle.dump(model, fh)
    print('File saved: ',path+filename+'.'+ext)
    print('Time completed:', datetime.now())

In [12]:
from pypdf import PdfReader
import pandas as pd
import openai
import os
from openai.embeddings_utils import get_embedding, cosine_similarity
import re

In [22]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Create prompt variants

In [30]:
from itertools import product
task_part1 = [
    "Summarize the article",
    "Summarize the article for a Tiktok post"
]
audience = [
    "lay audience",
    # "seniors"
]
task_part2 = [
    "",
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]
combinations = pd.DataFrame(product(task_part1, task_part2, audience), 
    columns=['task part 1', 'task part 2', 'audience'])


combinations['prompt'] = combinations.apply(
    lambda row: row['task part 1'] + ' for ' + row['audience'] + '. ' + row['task part 2'], 
    axis=1)
combinations

,task part 1,task part 2,audience,prompt
0,Summarize the article,,lay audience,Summarize the article for lay audience.
1,Summarize the article for a Tiktok post,,lay audience,Summarize the article for a Tiktok post for lay audience.


# Iteration 1

In [35]:
env_name = 'api_openai'

class Vectorize_PDF:
    # def __init__(self, system_role, model, temperature, n_choices, max_tokens):
    #     self.system_role = system_role
    #     self.temperature = temperature
    #     self.n_choices = n_choices
    #     self.max_tokens = max_tokens
    #     self.model = model
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page = pdf.pages[i]
            page_text = []

            def visitor_body(text, cm, tm, fontDict, fontSize):
                x = tm[4]
                y = tm[5]
                # ignore header/footer
                if (y > 50 and y < 720) and (len(text.strip()) > 1):
                    page_text.append({
                    'fontsize': fontSize,
                    'text': text.strip().replace('\x03', ''),
                    'x': x,
                    'y': y
                    })

            _ = page.extract_text(visitor_text=visitor_body)

            blob_font_size = None
            blob_text = ''
            processed_text = []

            for t in page_text:
                if t['fontsize'] == blob_font_size:
                    blob_text += f" {t['text']}"
                    if len(blob_text) >= 2000:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                        blob_font_size = None
                        blob_text = ''
                else:
                    if blob_font_size is not None and len(blob_text) >= 1:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                    blob_font_size = t['fontsize']
                    blob_text = t['text']
                paper_text += processed_text
        print("Done parsing paper")
        return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 30:
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df
    
    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv(env_name)
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

class Chatbot:
    def __init__(self, system_role, model, temperature, n_choices, max_tokens):
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        # print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        # print(result)
        system_role = f"""{self.system_role} You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the language of the query. 
        Here is the question: """+ user_input + """
            
        Here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response

def process_pdf(pdf_filename, path, filename=None,
    save_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Extract the text from the saved PDF and create the embeddings using OpenAI.
    Parameters:
        - pdf_filename (str): Filename of PDF.
        - path (raw string): File path copied from File Explorer.
    Returns:
        - DataFrame containing parsed text and word embeddings.
        - PdfReader(filename) class object.
        - Chatbot() class object.
    """
    print("Processing PDF")
    full_path = f'{path}/'.replace('\\','/')+pdf_filename

    pdf = PdfReader(full_path)
    chatbot = Vectorize_PDF()
    paper_text = chatbot.extract_text(pdf)
    df = chatbot.create_df(paper_text)
    df = chatbot.embeddings(df)
    if filename:
        try:
            filename = filename if (type(filename) == str) else (re.sub(r'(.*).pdf', r'\1', pdf_filename))
            savepickle(chatbot, filename='chatgpt_embeddings_'+filename, path=save_path)
            savepickle(df, filename='embeddings_df_'+filename, path=save_path)
        except:
            print('Unable to save outputs')
    
    return df, pdf, chatbot


def reply(df, user_input, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=None,
        system_role = """
        whose expertise is reading and summarizing scientific papers.
        """):
    chatbot = Chatbot(
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(
        df, user_input
        )
    qna = dict()
    qna['system_role'] = system_role
    qna['prompt'] = user_input
    try:
        response = chatbot.gpt(prompt)
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        for index, source in enumerate(sources):
            qna[f'source_{index+1}'] = source
    except:
        print('**Unable to get response**')
    return qna, chatbot

pdf_filename = "Daily energy expenditure through the human life course.pdf"
path = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"
iteration = 1
df_dict = dict()
pdf_dict = dict()
chatbot_dict = dict()

df_dict[iteration], pdf_dict[iteration], chatbot_dict[iteration] = process_pdf(pdf_filename, path, filename=1)
iteration +=1

Processing PDF
Parsing paper
Total number of pages: 14
Done parsing paper
Creating dataframe
Done creating dataframe
Calculating embeddings
Done calculating embeddings
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatgpt_embeddings_Daily energy expenditure through the human life course.pdf.sav
Unable to save outputs


In [41]:
save_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
filename=None
filename = filename if (type(filename) == str) else (re.sub(r'(.*).pdf', r'\1', pdf_filename))
savepickle(df_dict[1], filename='embeddings_df_'+filename, path=save_path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/embeddings_df_Daily energy expenditure through the human life course.sav
Time completed: 2023-04-05 20:23:35.086794


In [59]:
print(len(pdf_dict[1].pages[2]))
pdf_dict[1].pages[2].extract_text()

10


'Juveniles (1 to 20 y):\nTotal and basal expenditure, along with fat free mass, continued to increase with age \nthroughout childhood and adolescence (Figure 1), but body size-adjusted expenditures \nsteadily declined. Adjusted total expenditure declined at a rate of −2.8 ± 0.1% per year \nfrom 147.8 ± 22.6% for subjects 1 – 2 y (n = 102) to 102.7 ± 18.1% for subjects 20 – \n25 y (n = 314; Tables S2, S4). Segmented regression analysis identified a breakpoint in \nadjusted total expenditure at 20.5 y (95% CI: 19.8, 21.2), after which it plateaued at adult \nlevels (Figure 2). A similar decline (−3.8 ± 0.2% per year) and break point (18.0 y, 95% \nCI: 16.8, 19.2) were evident in adjusted basal expenditure (Figure 2, Text S1, Table S4). No \npubertal increases in adjusted total or basal expenditure were evident among subjects 10 – \n15 y. In multivariate regression for subjects 1 to 20 y, males had a higher total expenditure \nand adjusted total expenditure (Tables S2, S3), but sex had no

In [61]:
chatbot_dict[1].extract_text(pdf_dict[1])

Parsing paper
Total number of pages: 14
Done parsing paper


[{'fontsize': 14.0,
  'text': 'Daily Energy Expenditure through the Human Life Course',
  'page': 0},
 {'fontsize': 14.0,
  'text': 'Daily Energy Expenditure through the Human Life Course',
  'page': 0},
 {'fontsize': 10.0,
  'text': 'A full list of authors and affiliations appears at the end of the article.',
  'page': 0},
 {'fontsize': 14.0,
  'text': 'Daily Energy Expenditure through the Human Life Course',
  'page': 0},
 {'fontsize': 10.0,
  'text': 'A full list of authors and affiliations appears at the end of the article.',
  'page': 0},
 {'fontsize': 12.0, 'text': 'Abstract', 'page': 0},
 {'fontsize': 14.0,
  'text': 'Daily Energy Expenditure through the Human Life Course',
  'page': 0},
 {'fontsize': 10.0,
  'text': 'A full list of authors and affiliations appears at the end of the article.',
  'page': 0},
 {'fontsize': 12.0, 'text': 'Abstract', 'page': 0},
 {'fontsize': 14.0,
  'text': 'Daily Energy Expenditure through the Human Life Course',
  'page': 0},
 {'fontsize': 10.0,


### Embeddings DataFrame

In [51]:
df_dict[1]

,fontsize,text,page,length,embeddings,similarity
0,14.0,Daily Energy Expenditure through the Human Life Course,0,54,"[0.011471190489828587, -0.00378715549595654, 0.0077937594614923, -0.025070369243621826, -0.006500341463834047, -0.0014189364155754447, -0.007155362982302904, 0.001191175077110529, -0.01779530756175518, 0.000975882459897548, -0.00704231346026063, 0.025722065940499306, 0.01584021747112274, 0.020096195861697197, -0.01262828428298235, 0.004841174930334091, 0.03646841272711754, -0.009589250199496746, 0.01800810731947422, -0.023700477555394173, -0.022357184439897537, 0.007620860356837511, -0.0021429513581097126, -0.021625688299536705, -0.00234411284327507, 0.003900204785168171, 0.03000464476644993, -0.03583001345396042, 0.012814483605325222, 0.003757230471819639, 0.010141197592020035, -0.0260146651417017, 0.00857845600694418, -0.026147665455937386, 0.007960008457303047, 0.0025253244675695896, -0.02428567409515381, 0.02274288237094879, 0.01408462692052126, -0.007388111669570208, 0.024126075208187103, 0.022782782092690468, -0.007241812534630299, 0.017848508432507515, 0.0017772032879292965, 0.018606603145599365, 0.0007368993246927857, -0.024232475087046623, -0.016904212534427643, -0.011544340290129185, 0.004418902099132538, 0.012581734918057919, -0.013938328251242638, 0.0025635617785155773, -0.014337325468659401, 0.007328262086957693, -0.019471099600195885, 0.008658255450427532, 0.010433795861899853, -0.029818445444107056, -0.01595991663634777, -0.018353905528783798, -0.02750425785779953, 0.021811887621879578, -0.018380504101514816, -0.011218491941690445, 0.0103539964184165, 0.01597321778535843, 0.022609882056713104, 0.009575949981808662, 0.025801867246627808, 0.0006425529136322439, -0.012601684778928757, 0.011504440568387508, 0.026254063472151756, -0.012229287065565586, -0.008319106884300709, -0.002196151064708829, 0.02513686940073967, 0.010912593454122543, 0.019377999007701874, -0.010733044706285, -0.016438715159893036, 0.010181097313761711, 0.0032917328644543886, -0.006477066315710545, -0.001047369558364153, 0.007906809449195862, 0.003913504537194967, -0.0054097468964755535, 0.011850238777697086, -0.014310725964605808, 0.010001548565924168, 0.052800726145505905, -0.008904303424060345, 0.002428899984806776, -0.01605301722884178, 0.0122891366481781, -0.004541926551610231, -0.021293189376592636, ...]",0.748200
2,10.0,A full list of authors and affiliations appears at the end of the article.,0,74,"[0.006952525582164526, 0.0001836171286413446, -0.0020156342070549726, -0.035679884254932404, -0.028793824836611748, 0.03557353466749191, -0.030973967164754868, 0.008527811616659164, -0.019461754709482193, 0.002116997493430972, 0.022346453741192818, 0.019980203360319138, 0.005310772452503443, 0.028022799640893936, -0.015779441222548485, -0.008388228714466095, 0.02767716720700264, -0.0034463524352759123, 0.010488609783351421, -0.029618024826049805, -0.014662783592939377, 0.021110154688358307, -0.030309289693832397, 0.014543142169713974, 0.007783372420817614, 0.0140778673812747, 0.018065931275486946, -0.01781335473060608, 0.010129683651030064, 0.01048196293413639, -0.019102828577160835, 0.013426483608782291, 0.0006950033130124211, -0.032170385122299194, -0.014476673677563667, 0.009425126016139984, -0.009072846733033657, -0.015088177286088467, 0.030575161799788475, 0.000685033155605197, 0.001925902790389955, 0.01821216195821762, -0.016736576333642006, -0.02388850599527359, -6.854486127849668e-05, -0.01137263048440218, 0.038152482360601425, -0.014436793513596058, -0.02501845732331276, 0.013526185415685177, 0.01293462235480547, -0.015035003423690796, -0.017481015995144844, -0.028660889714956284, -0.01113999355584383, -0.021947648376226425, -0.011625207960605621, 0.008873443119227886, -0.025098219513893127, -0.02513810060918331, -0.004366930574178696, -0.009205781854689121, -0.03506838157773018, -0.011857844889163971, -0.006769739557057619, -0.02863430231809616, -0.01593896374106407, 0.01132610347121954, 0.012914681807160378, -0.003039

In [44]:
combinations

,task part 1,task part 2,audience,prompt
0,Summarize the article,,lay audience,Summarize the article for lay audience.
1,Summarize the article for a Tiktok post,,lay audience,Summarize the article for a Tiktok post for lay audience.


In [45]:
for index, value in combinations['prompt'].items():
    print(index)

0
1


In [47]:
chat_data_dict = dict()
iteration = 1

In [50]:
system_role = "You are an expert at science communication. You provide summaries in under 300 characters."
chat_data_dict[iteration] = dict()
for index, prompt in combinations['prompt'].items():
    prompt_num = index + 1 
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(
        df_dict[1], prompt, max_tokens=1500, system_role=system_role
        )
    chat_data_dict[iteration][prompt_num] = response

iteration += 0.1
pd.DataFrame(chat_data_dict[iteration-0.1])

Done creating prompt
Sending request to GPT-3
	Requesting 5 choices
Done sending request to GPT-3
Done creating prompt
Sending request to GPT-3
	Requesting 5 choices
Done sending request to GPT-3


,1,2
system_role,You are an expert at science communication. You provide summaries in under 300 characters.,You are an expert at science communication. You provide summaries in under 300 characters.
prompt,Summarize the article for lay audience.,Summarize the article for a Tiktok post for lay audience.
response_01,"I'm sorry, but without the title or any information about the content of the paper, I cannot provide a detailed answer. Can you please provide me with the title of the paper or some context about the topic?","I'm sorry, but without the actual paper, I cannot provide a detailed summary. The given embeddings do not contain enough information to understand what the article is about. Can you provide me with the title or any additional information about the paper?"
response_02,"I'm sorry, but I cannot provide a detailed answer without knowing the title of the paper and the query for which the text embeddings were generated. Could you please provide me with that information?","I'm sorry but based on the embeddings provided, I cannot infer the topic or content of the paper. Can you please provide the title of the paper or some more context?"
response_03,"I'm sorry, but I cannot provide a detailed answer without knowing the title of the paper and the query for which the embeddings were generated. Can you please provide me with that information?","I'm sorry, I cannot generate a detailed answer without the actual paper or a specific query to work with. Could you please provide more information or context so I can assist you better?"
response_04,"I apologize, but I cannot provide a detailed answer based on the given text embeddings alone. The embeddings do not contain enough information about the actual content of the article. In order to provide a detailed summary, I would need the title or abstract of the article.","I'm sorry, but without any context or information about the paper, I cannot provide a detailed summary. Please provide me with the title or a brief overview of the paper so that I can generate an appropriate summary for a Tiktok post."
response_05,"I'm sorry, but without the title of the paper or the query for which the embeddings were generated, I cannot provide a summary. Please provide the title of the paper and the query for which the embeddings were generated.","I'm sorry, but without the title or context of the paper, I cannot provide a detailed summary for a Tiktok post for lay audience using the given embeddings. Could you please provide the title or topic of the paper?"
source_1,{'Page 0': 'A full list of authors and affiliations appears at the end of the article....'},{'Page 0': 'A full list of authors and affiliations appears at the end of the article....'}
source_2,{'Page 4': 'Refer to Web version on PubMed Central for supplementary material....'},{'Page 0': 'Science . Author manuscript; available in PMC 2022 August 13....'}
source_3,{'Page 0': 'Science . Author manuscript; available in PMC 2022 August 13....'},{'Page 4': 'Refer to Web version on PubMed Central for supplementary material....'}


For some reason, parsing is not per page, as seen in the embeddings dataframe

In [70]:
print(len(chat_data_dict[iteration-0.1][1]['source_1']['Page 0']))
chat_data_dict[iteration-0.1][1]['source_1']['Page 0']

77


'A full list of authors and affiliations appears at the end of the article....'

# Iteration 2: Improve embedding

In [77]:
env_name = 'api_openai'

class Vectorize_PDF:
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page = pdf.pages[i]
            page_text = []

            def visitor_body(text, cm, tm, fontDict, fontSize):
                x = tm[4]
                y = tm[5]
                # ignore header/footer
                if (y > 50 and y < 720) and (len(text.strip()) > 1):
                    page_text.append({
                    'fontsize': fontSize,
                    'text': text.strip().replace('\x03', ''),
                    'x': x,
                    'y': y
                    })

            _ = page.extract_text(visitor_text=visitor_body)

            blob_font_size = None
            blob_text = ''
            processed_text = []

            for t in page_text:
                if t['fontsize'] == blob_font_size:
                    blob_text += f" {t['text']}"
                    if len(blob_text) >= 2000:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                        blob_font_size = None
                        blob_text = ''
                else:
                    if blob_font_size is not None and len(blob_text) >= 1:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                    blob_font_size = t['fontsize']
                    blob_text = t['text']
                paper_text += processed_text
        print("Done parsing paper")
        return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 130: # SH 2023-04-05 21:11 Increased from 30 to 130
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df
    
    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv(env_name)
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

class Chatbot:
    def __init__(self, system_role, model, temperature, n_choices, max_tokens):
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        # print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        # print(result)
        system_role = f"""{self.system_role} You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the language of the query. 
        Here is the question: """+ user_input + """
            
        Here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response

def process_pdf(pdf_filename, path, filename=None, append_version=False,
    save_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Extract the text from the saved PDF and create the embeddings using OpenAI.
    Parameters:
        - pdf_filename (str): Filename of PDF.
        - path (raw string): File path copied from File Explorer.
    Returns:
        - DataFrame containing parsed text and word embeddings.
        - PdfReader(filename) class object.
        - Chatbot() class object.
    """
    print("Processing PDF")
    full_path = f'{path}/'.replace('\\','/')+pdf_filename

    pdf = PdfReader(full_path)
    chatbot = Vectorize_PDF()
    paper_text = chatbot.extract_text(pdf)
    df = chatbot.create_df(paper_text)
    df = chatbot.embeddings(df)
    if filename:
        try:
            filename = filename if (type(filename) == str) else (re.sub(r'(.*).pdf', r'\1', pdf_filename))
            savepickle(chatbot, filename='chatgpt_embeddings_'+filename, path=save_path)
            savepickle(df, filename='embeddings_df_'+filename, path=save_path)
        except:
            print('Unable to save outputs')
    
    return df, pdf, chatbot


def reply(df, user_input, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=None,
        system_role = """
        whose expertise is reading and summarizing scientific papers.
        """):
    chatbot = Chatbot(
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(
        df, user_input
        )
    qna = dict()
    qna['system_role'] = system_role
    qna['prompt'] = user_input
    try:
        response = chatbot.gpt(prompt)
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        for index, source in enumerate(sources):
            qna[f'source_{index+1}'] = source
    except:
        print('**Unable to get response**')
    return qna, chatbot


In [78]:

pdf_filename = "Daily energy expenditure through the human life course.pdf"
path = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"
iteration = 2

df_dict[iteration], pdf_dict[iteration], chatbot_dict[iteration] = process_pdf(
    pdf_filename, path, filename=1, append_version=True)
iteration +=1

Processing PDF
Parsing paper
Total number of pages: 14
Done parsing paper
Creating dataframe
Done creating dataframe
Calculating embeddings
Done calculating embeddings
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatgpt_embeddings_Daily energy expenditure through the human life course.sav
Time completed: 2023-04-05 21:14:56.817121
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/embeddings_df_Daily energy expenditure through the human life course.sav
Time completed: 2023-04-05 21:14:56.825122


In [80]:
df_dict[2]

,fontsize,text,page,length,embeddings
0,10.0,"Total daily energy expenditure (“total expenditure”, MJ/d) reflects daily energy needs and is a critical variable in human health and physiology, yet it is unclear how daily expenditure changes over the life course. Here, we analyze a large, globally diverse database of total expenditure measured by the doubly labeled water method for males and females aged 8 days to 95 yr. We show that total expenditure is strongly related to fat free mass in a power-law manner and identify four distinct metabolic life stages. Fat free mass-adjusted daily expenditure accelerates rapidly in neonates (0-1yr) to ~46% above adult values at ~1 yr, declines slowly throughout childhood and adolescence (1-20 yr) to adult levels at ~20 yr, remains stable in adulthood (20-60 yr) even during pregnancy, and declines in older adults (60+ yr). These changes in total expenditure shed new light on human development and aging and should help shape nutrition and health strategies across the lifespan.",0,981,"[0.007350415922701359, 0.0017239886801689863, -0.006178037729114294, -0.03071894310414791, 0.0018326642457395792, -0.0021883295848965645, -0.014476894401013851, -0.01564927212893963, -0.029217245057225227, -0.015043324790894985, 0.0026345576625317335, 0.02776823751628399, 0.009385611861944199, 0.017177315428853035, -0.025858184322714806, 0.003140063490718603, 0.03904414549469948, -0.001225892687216401, 0.011098074726760387, -0.01081485953181982, -0.006013378035277128, 0.01384460087865591, -0.012375835329294205, -0.00977420900017023, 0.0032882574014365673, 0.0024386125151067972, 0.011229801923036575, -0.01859997771680355, -0.0018491301452741027, -0.016558194532990456, 0.022275185212492943, -0.012724913656711578, -0.005519398488104343, -0.016742615029215813, -2.352834962948691e-05, 0.008687454275786877, -0.01974600926041603, -0.011960892006754875, 0.03156200051307678, -0.006856436375528574, 0.014937941916286945, 0.024172067642211914, -0.005993618629872799, 0.02465946041047573, -0.007949777878820896, 3.3369349694112316e-05, 0.012309971265494823, -0.03129854425787926, -0.0009780795080587268, 0.01180281862616539, -0.006151692476123571, 0.02004898339509964, -0.020444167777895927, 0.000139446317916736, -0.025845011696219444, -0.0027893378864973783, -0.003793763229623437, 0.022117111831903458, 0.016228875145316124, -0.02255181409418583, -0.018455076962709427, -0.010426262393593788, -0.03245775029063225, 0.009945455007255077, 0.0003869506763294339, -0.02369784750044346, -0.00708037381991744, 0.016637232154607773, 0.02057589590549469, 0.012929092161357403, 0.018744878470897675, 0.022578159347176552, -0.009299988858401775, 0.009359266608953476, 0.02559472806751728, -0.004689512774348259, 0.006481011863797903, -0.0015107542276382446, 0.01045260764658451, -0.014964288100600243, 0.024580422788858414, -0.017203662544488907, -0.01912688836455345, 0.023645155131816864, 0.007403107360005379, -0.011420807801187038, -0.0009904290782287717, 0.0023892144672572613, -0.005631367210298777, 0.0054667070508003235, 0.0033113097306340933, -0.015833692625164986, 0.028453223407268524, 0.04065122455358505, 0.0009706698474474251, -0.0010933415032923222, -0.01012987457215786, 0.011322012171149254, -0.01890295185148716, -0.016373775899410248, ...]"
24,10.0,"Expenditure varies as we age, with four distinct metabolic life stages reflecting changes in behavior, anatomy, and tissue metabolism. All of life’s essential tasks, from development and reproduction to maintenance and movement, require energy. Total expenditure is thus fundamental to understanding both daily nutritional requirements and the body’s investment among activities. Yet we know surprisingly little about the determinants of total expenditure in humans or how it changes over the lifespan. Most large (n>1,000) analyses of human energy expenditure have been limited to basal expenditure, the metabolic rate at rest ( ), which accounts for only a portion (usually ~50-70%) of total expendi

In [83]:
system_role = "You are an expert at science communication. You provide summaries in under 300 characters."
chat_data_dict[iteration-1] = dict()
for index, prompt in combinations['prompt'].items():
    prompt_num = index + 1 
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(
        df_dict[iteration-1], prompt, max_tokens=1500, system_role=system_role
        )
    chat_data_dict[iteration-1][prompt_num] = response

iteration += 0.1
pd.DataFrame(chat_data_dict[iteration-0.1])

Done creating prompt
Sending request to GPT-3
	Requesting 5 choices
Done sending request to GPT-3
Done creating prompt
Sending request to GPT-3
	Requesting 5 choices
Done sending request to GPT-3


KeyError: 3.0

In [84]:
pd.DataFrame(chat_data_dict[2])

,1,2
system_role,You are an expert at science communication. You provide summaries in under 300 characters.,You are an expert at science communication. You provide summaries in under 300 characters.
prompt,Summarize the article for lay audience.,Summarize the article for a Tiktok post for lay audience.
response_01,"This article explores how our daily energy needs change throughout our lives. Using a large database, the authors found that our energy needs are strongly related to our muscle mass, and identified four stages of life where our energy needs change. Babies need the most energy, whereas older adults need the least. These findings have important implications for our nutrition and health strategies throughout our lives.","I'm sorry, but you haven't provided me with the title or topic of the paper. Could you please provide that information so I can give you an accurate summary for a Tiktok post?"
response_02,"This paper analyzed a large database of energy expenditure in people aged 8 days to 95 years. They found that daily energy needs are strongly related to muscle mass and identified four life stages with distinct patterns of energy expenditure. These findings shed new light on human development and aging, and could inform nutrition and health strategies across the lifespan.","I'm sorry, but you haven't provided the title or any information about the paper. Please provide that information so I can create a detailed summary for a Tiktok post."
response_03,"This study analyzed a large database of daily energy expenditure across different ages and genders. The results showed that daily energy needs are strongly related to muscle mass, with four distinct stages of metabolic life. Daily energy expenditure accelerates rapidly in neonates, declines throughout childhood and adolescence before stabilizing in adulthood, and declines in older adults. These findings provide insights into human development and aging and should guide nutrition and health strategies across the lifespan.","I'm sorry, but I need more information to provide an accurate summary. What is the title of the paper and what is the query that the embeddings are based on?"
response_04,"This article explores how our daily energy expenditure changes over the course of our lives. The study analyzed a large and diverse sample of individuals from 8 days to 95 years old. Results showed that our energy expenditure is strongly related to our fat-free mass, and that we go through four distinct metabolic life stages. These stages include a rapid increase in energy expenditure during infancy, a slow decline throughout childhood and adolescence, stability during adulthood (including pregnancy), and a decline in older age. Understanding these changes can help shape nutrition and health strategies throughout our lifespan.","I'm sorry, I cannot provide a detailed answer without knowing the title and subject of the paper. The embeddings only provide information about the affiliations of the authors and the institution where the research was conducted. Please provide more information about the paper or the topic you are interested in."
response_05,"The paper analyzes how much energy people need throughout their lives. They studied a large group of people of all ages and found that the amount of energy needed varies depending on age and muscle mass. Babies need the most energy, while older adults need less. This information is important for creating healthy nutrition and exercise plans.","I'm sorry, I don't have enough information to provide a detailed answer. Please provide the title or a brief summary of the paper so I can generate a Tiktok post summary."
source_1,"{'Page 6': 'Nutritional and Health Related Environmental Studies Section, Division of Human Health, International Atomic Energy Agency, Vienna, Austria...'}","{'Page 6': 'Center for Energy Metabolism and Reproduction, Shenzhen Institutes of Advanced Technology, Chinese Academy of Sciences, Shenzhen, China...'}"
sourc

In [87]:
df_dict[2][df_dict[2]['page'] == 6]

,fontsize,text,page,length,embeddings,similarity
512,10.0,"Division of Public Health Sciences, Fred Hutchinson Cancer Research Center and School of Public Health, University of Washington, Seattle, WA, USA",6,146,"[0.0027888002805411816, 0.0009523712797090411, -0.0038930687587708235, -0.014389244839549065, -0.01098160445690155, 0.003084557829424739, -0.0034719358664005995, 0.0025605526752769947, -0.0010544398101046681, 0.001994755817577243, -0.0193785447627306, -0.0023532009217888117, 0.034256432205438614, -0.003375493222847581, -0.0053622121922671795, -0.0035233718808740377, 0.03500225767493248, -0.024882206693291664, -0.007393937557935715, -0.01908278837800026, -0.03816557675600052, 0.006410222500562668, 0.011026610620319843, -0.017102498561143875, -0.009972170926630497, 0.03543946519494057, 0.000771943130530417, -0.029781494289636612, 0.01604805886745453, -0.0148650286719203, -0.0063716452568769455, 0.004381711594760418, -0.021371694281697273, -0.00826835073530674, -0.01383630745112896, 0.00332084228284657, 0.0015141498297452927, 0.006169115658849478, 0.0009234384633600712, -0.018092643469572067, -0.020111510530114174, -0.01186244748532772, -0.008615544065833092, -0.016189508140087128, -0.002499472349882126, 0.003132779151201248, 0.012434673495590687, -0.019918624311685562, -0.027441151440143585, 0.028032667934894562, 0.017462551593780518, 0.009316361509263515, -0.02213037572801113, -0.011328797787427902, -0.019172800704836845, -0.01217106357216835, 0.0034494325518608093, 0.02052299864590168, -0.020098650828003883, -0.003426929237321019, -0.0012521472526714206, -0.009560682810842991, -0.041586074978113174, 0.018182655796408653, -0.0209859237074852, -0.003732331097126007, -0.002478576498106122, 0.016189508140087128, 0.005571171175688505, 0.01989290677011013, 0.021577438339591026, 0.01098160445690155, -0.008866295218467712, -0.011965319514274597, 0.025538016110658646, 5.575591421802528e-05, -0.016626715660095215, 0.01487788837403059, -0.007258917670696974, -0.00970213208347559, 0.014093487523496151, -0.017886899411678314, -0.013501972891390324, 0.009714990854263306, 0.014054910279810429, 0.005661184433847666, 0.017063921317458153, 0.014029192738234997, 0.0011131090577691793, -0.010480103082954884, 0.01652384363114834, 0.025859491899609566, 0.00533649418503046, -0.006821711082011461, -0.015405108220875263, 0.002695572329685092, 0.011026610620319843, 0.03361348435282707, 0.016948189586400986, -0.03865421935915947, ...]",0.720459
557,10.0,"Maastricht University, Maastricht and Lifestyle Medicine Center for Children, Jeroen Bosch Hospital’s-Hertogenbosch, The Netherlands",6,132,"[0.01477845385670662, 0.0004291976219974458, -5.469550524139777e-05, -0.03657131642103195, 0.014684749767184258, 0.03276961296796799, 0.015407608821988106, 0.0049562701024115086, 0.0019426839426159859, -0.003211033996194601, -0.018205340951681137, -0.004614919889718294, 0.0026973355561494827, -0.005776179954409599, 0.002210409613326192, -0.019798308610916138, 0.03691935911774635, -0.005869884043931961, 0.008714468218386173, -0.016947031021118164, -0.008085313253104687, 0.029851404950022697, -0.0019041983177885413, -0.0033164508640766144, 0.00469523761421442, -0.0021652309224009514, 0.028298595920205116, -0.02861986681818962, 0.0023024401161819696, 0.01173307467252016, 0.00016272696666419506, -0.006686447188258171, -0.02080227993428707, -0.019195925444364548, 0.01186024397611618, 0.009551110677421093, 0.011612597852945328, 0.00628485856577754, 0.020614871755242348, -0.010334208607673645, 0.0027358210645616055, -0.003962338902056217, 0.005515147466212511, -0.01265003439038992, -0.02177947759628296, 0.012315377593040466, 0.018098250031471252, -0.0055553060956299305, -0.02784346230328083, 0.003112310077995062, 0.0009713419713079929, 0.035125598311424255, -0.018700633198022842, -0.005903349723666906, -0.011030294932425022, -0.00533777941018343, 0.008158938027918339, 0.013419745489954948, 0.004327115137130022, -0.0348043292760849, -0.006840389221906662, -0.0228

In [86]:
df_dict[2][df_dict[2]['page'] == 5]

,fontsize,text,page,length,embeddings,similarity
299,10.0,"Institute for Active Health, Kyoto University of Advanced Science, Kyoto, Japan National Institute of Health and Nutrition, National Institutes of Biomedical Innovation, Health and Nutrition, Tokyo, Japan Faculty of Health and Sport Sciences, University of Tsukuba, Ibaraki, Japan Research Institute for Sport and Exercise Sciences, Liverpool John Moores University, Liverpool, UK Department of Nutrition, Institute of Basic Medical Sciences, University of Oslo, 0317 Oslo, Norway Crewe Alexandra Football Club, Crewe, UK David Geffen School of Medicine, University of California, Los Angeles",5,592,"[-0.0007704428280703723, 0.011743593961000443, -0.00384299224242568, -0.016780395060777664, -0.006787274498492479, 0.017880309373140335, -0.005261478945612907, 0.0067000859417021275, 0.008658469654619694, -0.015841444954276085, 0.0020606620237231255, -0.025606535375118256, 0.017088908702135086, -0.0012474635150283575, -0.015318314544856548, -0.025069991126656532, 0.037182457745075226, -0.023259157314896584, 0.00624737748876214, -0.018255889415740967, -0.027578331530094147, 0.022816509008407593, -0.002709544263780117, -0.012575236149132252, -0.012420980259776115, 0.023205503821372986, 0.0024546862114220858, -0.02250799722969532, -0.00016022947966121137, -0.018255889415740967, 0.012213069945573807, -0.007089079823344946, -0.0246944110840559, -0.012434394098818302, 0.007531728595495224, 0.011810662224888802, -0.02811487577855587, -0.017531557008624077, 0.015747549012303352, -0.022387273609638214, 0.00305829755961895, 0.008577988483011723, -0.005174290854483843, 0.001400043023750186, -0.007652450818568468, 0.00016599312948528677, 0.009423043578863144, 0.0009959586896002293, -0.012783147394657135, 0.04909372329711914, 0.013909888453781605, 0.03618985414505005, -0.009496818296611309, -0.015117110684514046, -0.020710576325654984, -0.004372829105705023, -0.015090283937752247, 0.029992777854204178, -0.017209630459547043, -0.012937403284013271, -0.00976509042084217, -0.018966810777783394, -0.03385588899254799, 0.020952021703124046, 0.007779879495501518, 0.0016540627693757415, -0.02278968133032322, 0.0061333621852099895, -0.012749613262712955, 0.014406191185116768, 0.01329286303371191, 0.021032502874732018, -0.02549922652542591, 0.009241960942745209, 0.03356079012155533, -0.011783835478127003, -0.013909888453781605, -0.002292046556249261, 0.00793413631618023, 0.0006241509108804166, 0.019355803728103638, -0.02285674959421158, -0.0032460878137499094, -0.010838177055120468, 0.023285984992980957, 0.0003259082150179893, 0.005466036032885313, 0.02366156503558159, 0.01231367141008377, -0.008282889612019062, 0.009040757082402706, 0.03181702643632889, 0.0023054599296301603, 0.009852278977632523, -0.04311126470565796, -0.0066531384363770485, 0.007015305105596781, 0.029161134734749794, 0.002726311329752207, -0.01332639716565609, ...]",0.714477
304,10.0,"Unité Mixte de Recherche en Nutrition et Alimentation, CNESTEN- Université Ibn Tofail URAC39, Regional Designated Center of Nutrition Associated with AFRA/IAEA",5,159,"[-0.0012074422556906939, -0.01324793417006731, 0.006425347179174423, -0.013665035367012024, 0.0039492216892540455, 0.012691798619925976, -0.001467303023673594, -0.008606853894889355, -0.014578686095774174, -0.004280254244804382, 0.007554169278591871, -0.01424765307456255, 0.006498174276202917, -0.027462482452392578, -0.022960437461733818, -0.008236097171902657, 0.02274857647716999, -0.01819356344640255, -0.002115299692377448, -0.0012082698522135615, -0.01810087449848652, 0.014671375043690205, 0.00795802939683199, -0.004498736001551151, 0.0068060350604355335, 0.0003463430912233889, 0.009606572799384594, -0.032308805733919144, 0.025396838784217834, -0.006332657765597105, 0.020722655579447746, -0.01565123163163662, 0.004978733602911234, -0.03167322278022766, 0.0072297570295631886, 0.010314983315765858, 0.005508386064320803, -0.01502889022231102, 0.03802905231714249, -0.028151031583547592, 0

summaries are poor due to search results returning affiliations as sources.

In [89]:
pdf_filename

'Daily energy expenditure through the human life course.pdf'

## Article 2 "Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf"

In [90]:
df_dict_sleep = dict()
pdf_dict_sleep = dict()
chatbot_dict_sleep = dict()
pdf_filename = "Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf"
path = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"
iteration = 2

df_dict_sleep[iteration], pdf_dict_sleep[iteration], chatbot_dict_sleep[iteration] = process_pdf(
    pdf_filename, path, filename=1, append_version=True)


Processing PDF
Parsing paper
Total number of pages: 10
Done parsing paper
Creating dataframe
Done creating dataframe
Calculating embeddings
Done calculating embeddings
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatgpt_embeddings_Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.sav
Time completed: 2023-04-05 21:35:24.712790
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/embeddings_df_Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.sav
Time completed: 2023-04-05 21:35:24.714791


In [91]:
df_dict_sleep[iteration]

,fontsize,text,page,length,embeddings
0,1.0,"ARTICLE Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions Neli Tsereteli1 Raphael Vallat2 Juan Fernandez-Tajes1 Linda M. Delahanty3 Jose M. Ordovas4,5 David A. Drew6,7 Ana M. Valdes8 Nicola Segata9,10 Andrew T. Chan6,7 Jonathan Wolf11 Sarah E. Berry12 Matthew P. Walker2 Timothy D. Spector13 Paul W. Franks1,14 Received: 23 April 2021 / Accepted: 3 August 2021 The Author(s) 2021 Abstract Aims/hypothesis Sleep, diet and exercise are fundamental to metabolic homeostasis. In this secondary analysis of a repeated measures, nutritional intervention study, we tested whether an individual s sleep quality, duration and timing impact glycaemic response to a breakfast meal the following morning. Methods Healthy adults data ( = 953 [41% twins]) were analysed from the PREDICT dietary intervention trial. Participants consumed isoenergetic standardised meals over 2 weeks in the clinic and at home. Actigraphy was used to assess sleep variables (duration, efficiency, timing) and continuous glucose monitors were used to measure glycaemic variation (>8000 meals). Results Sleep variables were significantly associated with postprandial glycaemic control (2 h incremental AUC), at both between- and within-person levels. Sleep period time interacted with meal type, with a smaller effect of poor sleep on postpran- dial blood glucose levels when high-carbohydrate (low fat/protein) ( pinteraction =0 . 0 2 ) a n d h i g h - f a t ( pinteraction = 0.03) breakfasts were consumed compared with a reference 75 g OGTT. Within-person sleep period time had a similar interaction (high carbohydrate: pinteraction =0 . 0 0 1 , h i g h f a t : pinteraction = 0.02). Within- and between-person sleep efficiency were significantly associated with lower postprandial blood glucose levels irrespective of meal type (both < 0.03). Later sleep midpoint (time deviation from midnight) was found to be significantly associated with higher postprandial glucose, in both between-person and",0,2017,"[0.0028571460861712694, -0.004616688936948776, 0.04285043850541115, -0.032853804528713226, -0.008010816760361195, 0.0013103695819154382, 0.009719700552523136, -0.023073313757777214, -0.02168188989162445, -0.018183067440986633, 0.006018244195729494, 0.0364471897482872, -0.006166842766106129, -0.00883486308157444, -0.011921661905944347, -0.007274577859789133, 0.037662994116544724, -0.017088841646909714, -0.0012284715194255114, 0.018412720412015915, -0.024680878967046738, 0.008848371915519238, -0.00631206389516592, 0.023235419765114784, -0.0006353435455821455, 0.009429257363080978, -0.01614321395754814, -0.010564010590314865, 0.007632565218955278, -0.008436349220573902, 0.011577183380723, -0.019128695130348206, -0.02250593714416027, 0.007200278341770172, 0.01325229462236166, -0.0393921434879303, -0.0007332835230045021, 0.009186096489429474, 0.03444785997271538, -0.0178318340331316, 0.030476225540041924, 0.012475529685616493, -0.00982101820409298, 0.004191156476736069, -0.007288086693733931, 0.018358683213591576, 0.046281713992357254, -0.00940899457782507, -0.004025671631097794, 0.030908511951565742, -0.006045261863619089, 0.0017916265642270446, -0.017656218260526657, -0.0027811583131551743, -0.0023184760939329863, -0.00606214813888073, -0.001089160330593586, 0.028287773951888084, -0.00680514145642519, -0.021155038848519325, -0.011158405803143978, -0.012110787443816662, -0.01908816769719124, 0.0009523820481263101, -0.007355631794780493, 0.0033012533094733953, -0.008220205083489418, 0.003004056168720126, -0.004380282014608383, 0.009138815104961395, 0.031637996435165405, 0.04039180651307106, -0.009800754487514496, 0.004370150156319141, 0.02809864841401577, -0.013454929925501347, -0.01169200986623764, -0.010894980281591415, -0.0012377589009702206, -0.0019638659432530403, 0.004579539410769939, -0.005227969493716955, -0.026045285165309906, 0.0359068289399147, -0.002723745070397854, -0.0009253641474060714

In [93]:
system_role = "You are an expert at science communication. You provide summaries in under 300 characters."
chat_data_dict_sleep = dict()
chat_data_dict_sleep[iteration] = dict()
for index, prompt in combinations['prompt'].items():
    prompt_num = index + 1 
    response, chatbot_dict_sleep[f'{iteration}_prompt{prompt_num}'] = reply(
        df_dict_sleep[iteration], prompt, max_tokens=1500, system_role=system_role
        )
    chat_data_dict_sleep[iteration][prompt_num] = response

pd.DataFrame(chat_data_dict_sleep[iteration])

Done creating prompt
Sending request to GPT-3
	Requesting 5 choices
Done sending request to GPT-3
Done creating prompt
Sending request to GPT-3
	Requesting 5 choices
Done sending request to GPT-3


,1,2
system_role,You are an expert at science communication. You provide summaries in under 300 characters.,You are an expert at science communication. You provide summaries in under 300 characters.
prompt,Summarize the article for lay audience.,Summarize the article for a Tiktok post for lay audience.
response_01,"This paper examines the relationship between sleep patterns and postprandial glucose metabolism in healthy adults. The study found that sleep duration, quality, and midpoint are important factors that can affect glucose levels after meals. Specifically, the study suggests that earlier bedtime routines and high-quality sleep can improve postprandial glucose metabolism. These findings may inform lifestyle strategies to improve metabolic health and prevent diabetes. However, the study has limitations, such as a small sample size and the use of a wearable device to measure glucose levels. Further studies are needed to confirm these findings and explore the effects of sleep stages on glucose metabolism.",The article studied how sleep and meal timing affect blood sugar levels in healthy individuals. They used a wearable device to monitor glucose levels and found that irregular sleep patterns and meal timings can lead to poor glucose tolerance. This has important implications for managing diabetes and overall health.
response_02,"The study found that getting enough good quality sleep at the right time can improve how our bodies process food, specifically glucose. The researchers used a wearable device to monitor participants' sleep and glucose levels after meals. They found that people who slept better and had a more consistent sleep schedule had better glucose metabolism. This suggests that establishing good sleep habits could be an important part of preventing diabetes and other metabolic disorders.",This article explores how sleep patterns and circadian rhythms affect glucose tolerance. The study used wearable devices to monitor participants' sleep and glucose levels after meals. Results showed that irregular sleep patterns and circadian misalignment can both impact glucose tolerance in different ways. This research could help improve understanding of the link between sleep and metabolic health.
response_03,"This study found that getting enough quality sleep and going to bed earlier may help improve blood sugar levels after meals. The study used a wearable device to monitor participants' sleep and blood glucose levels. They found that longer sleep duration and higher sleep quality were associated with lower blood glucose levels after meals, while later midpoint of sleep was associated with higher blood glucose levels. These findings suggest that good sleep habits may be an important part of preventing diabetes.",This study looked at how sleep and meal timing affect glucose levels. They monitored participants' sleep and used a device to measure their blood glucose levels after meals. They found that poor sleep and eating later in the day can both negatively impact glucose levels. This information could help people make better choices for their health.
response_04,The study suggests that getting enough good quality sleep and going to bed earlier can help improve glucose metabolism after meals in healthy adults. The researchers used wearable devices to monitor sleep and glucose levels in participants after they ate different types of meals. They found that people who slept longer and had better quality sleep had lower levels of blood sugar after meals. The findings suggest that improving sleep habits may be a useful strategy for managing blood sugar levels and preventing diabetes.,This article looks at how sleep and circadian rhythms affect glucose tolerance. The study found that irregular sleep patterns and being awake at night negatively impact glucose tolerance. The researchers used continuous glucose monitoring and found that postprandial blood glucose was higher in those with circadian misalignment. This study highlig

### Save CSV

In [104]:
{1: {'article': pdf_filename}, 2: {'article': pdf_filename}}

{1: {'article': 'Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf'},
 2: {'article': 'Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf'}}

In [107]:
{1: {'article': pdf_filename},**chat_data_dict_sleep[2][1]}

{1: {'article': 'Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf'},
 'system_role': 'You are an expert at science communication. You provide summaries in under 300 characters.',
 'prompt': 'Summarize the article for lay audience. ',
 'response_01': 'This paper examines the relationship between sleep patterns and postprandial glucose metabolism in healthy adults. The study found that sleep duration, quality, and midpoint are important factors that can affect glucose levels after meals. Specifically, the study suggests that earlier bedtime routines and high-quality sleep can improve postprandial glucose metabolism. These findings may inform lifestyle strategies to improve metabolic health and prevent diabetes. However, the study has limitations, such as a small sample size and the use of a wearable device to measure glucose levels. Further studies are needed to confirm these findings and explore the effects of sleep stages on gluc

In [109]:
chat_data_dict_sleep[2][2]

{'system_role': 'You are an expert at science communication. You provide summaries in under 300 characters.',
 'prompt': 'Summarize the article for a Tiktok post for lay audience. ',
 'response_01': 'The article studied how sleep and meal timing affect blood sugar levels in healthy individuals. They used a wearable device to monitor glucose levels and found that irregular sleep patterns and meal timings can lead to poor glucose tolerance. This has important implications for managing diabetes and overall health.',
 'response_02': "This article explores how sleep patterns and circadian rhythms affect glucose tolerance. The study used wearable devices to monitor participants' sleep and glucose levels after meals. Results showed that irregular sleep patterns and circadian misalignment can both impact glucose tolerance in different ways. This research could help improve understanding of the link between sleep and metabolic health.",
 'response_03': "This study looked at how sleep and meal t

In [127]:
to_export = dict()
to_export[1] = {**{'article': pdf_filename},**chat_data_dict_sleep[2][1]}
to_export

{1: {'article': 'Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf',
  'system_role': 'You are an expert at science communication. You provide summaries in under 300 characters.',
  'prompt': 'Summarize the article for lay audience. ',
  'response_01': 'This paper examines the relationship between sleep patterns and postprandial glucose metabolism in healthy adults. The study found that sleep duration, quality, and midpoint are important factors that can affect glucose levels after meals. Specifically, the study suggests that earlier bedtime routines and high-quality sleep can improve postprandial glucose metabolism. These findings may inform lifestyle strategies to improve metabolic health and prevent diabetes. However, the study has limitations, such as a small sample size and the use of a wearable device to measure glucose levels. Further studies are needed to confirm these findings and explore the effects of sleep stages on gl

In [128]:
to_export[2] = {**{'article': pdf_filename},**chat_data_dict_sleep[2][2]}
to_export

{1: {'article': 'Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf',
  'system_role': 'You are an expert at science communication. You provide summaries in under 300 characters.',
  'prompt': 'Summarize the article for lay audience. ',
  'response_01': 'This paper examines the relationship between sleep patterns and postprandial glucose metabolism in healthy adults. The study found that sleep duration, quality, and midpoint are important factors that can affect glucose levels after meals. Specifically, the study suggests that earlier bedtime routines and high-quality sleep can improve postprandial glucose metabolism. These findings may inform lifestyle strategies to improve metabolic health and prevent diabetes. However, the study has limitations, such as a small sample size and the use of a wearable device to measure glucose levels. Further studies are needed to confirm these findings and explore the effects of sleep stages on gl

In [132]:
pdf_filename1 = "Daily energy expenditure through the human life course.pdf"
to_export[3] = {**{'article': pdf_filename1},**chat_data_dict[2][1]}
to_export[4] = {**{'article': pdf_filename1},**chat_data_dict[2][2]}
pd.DataFrame(to_export)

,1,2,3,4
article,Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf,Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions.pdf,Daily energy expenditure through the human life course.pdf,Daily energy expenditure through the human life course.pdf
system_role,You are an expert at science communication. You provide summaries in under 300 characters.,You are an expert at science communication. You provide summaries in under 300 characters.,You are an expert at science communication. You provide summaries in under 300 characters.,You are an expert at science communication. You provide summaries in under 300 characters.
prompt,Summarize the article for lay audience.,Summarize the article for a Tiktok post for lay audience.,Summarize the article for lay audience.,Summarize the article for a Tiktok post for lay audience.
response_01,"This paper examines the relationship between sleep patterns and postprandial glucose metabolism in healthy adults. The study found that sleep duration, quality, and midpoint are important factors that can affect glucose levels after meals. Specifically, the study suggests that earlier bedtime routines and high-quality sleep can improve postprandial glucose metabolism. These findings may inform lifestyle strategies to improve metabolic health and prevent diabetes. However, the study has limitations, such as a small sample size and the use of a wearable device to measure glucose levels. Further studies are needed to confirm these findings and explore the effects of sleep stages on glucose metabolism.",The article studied how sleep and meal timing affect blood sugar levels in healthy individuals. They used a wearable device to monitor glucose levels and found that irregular sleep patterns and meal timings can lead to poor glucose tolerance. This has important implications for managing diabetes and overall health.,"This paper explores how our daily energy needs change throughout our lives. By analyzing a large database of people aged 8 days to 95 years old, the authors found that our total daily energy expenditure is strongly related to our fat-free mass. They identified four distinct stages of metabolic development, showing that our energy needs accelerate rapidly in our first year of life, decrease slowly throughout childhood and adolescence, remain stable during adulthood, and decline in older age. This information can help shape nutrition and health strategies for people of all ages.","I'm sorry, but you haven't provided me with the title or any information about the content of the paper. Please provide me with more details so that I can give you a proper summary."
response_02,"The study found that getting enough good quality sleep at the right time can improve how our bodies process food, specifically glucose. The researchers used a wearable device to monitor participants' sleep and glucose levels after meals. They found that people who slept better and had a more consistent sleep schedule had better glucose metabolism. This suggests that establishing good sleep habits could be an important part of preventing diabetes and other metabolic disorders.",This article explores how sleep patterns and circadian rhythms affect glucose tolerance. The study used wearable devices to monitor participants' sleep and glucose levels after meals. Results showed that irregular sleep patterns and circadian misalignment can both impact glucose tolerance in different ways. This research could help improve understanding of the link between sleep and metabolic health.,"This study analyzed a large database of total daily energy expenditure across different ages and found that it strongly relates to fat-free mass. The study identified four metabolic life stages, starting with a rapid increase in expenditure in neonates, followed by a slow decline throughout childhood and adolescence, stability in adulthood, and decline in older a

In [142]:
path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output'
# filename = 'prompt_engineering_experimentation_Daily energy expenditure through the human life course AND Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions'

filename = 'prompt_engineering_experimentation_Daily energy expenditure through the human life course'
savepickle(pd.DataFrame(to_export), filename=filename, path=path)#, append_version=True)
save_csv(pd.DataFrame(to_export), filename=filename, path=path, append_version=True)

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/prompt_engineering_experimentation_Daily energy expenditure through the human life course.sav
Time completed: 2023-04-05 22:31:36.746030
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/prompt_engineering_experimentation_Daily energy expenditure through the human life course2023-04-05_2231.csv
Time completed: 2023-04-05 22:31:36.904079


* Conclusion is included as source for first prompt, but not for second prompt.
* Abstract is not used for either prompt.

Next step:
* Include more embeddings in the prompt.

# *End of Page*